<a href="https://colab.research.google.com/github/ImagingDataCommons/CloudSegmentator/blob/main/workflows/TotalSegmentator/Notebooks/dicomSEGMaps/slicerMappingsTotalSegmentator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook generates the config map used for creating DICOM SEG files for segmentations created by TotalSegmentator
- The snomed codes for label and label maps are downloaded from official TotalSegmentator repo

###**Importing Packages**

In [31]:
import pandas as pd
import json
import yaml
import os
import sys
from pathlib import Path
import time
import numpy as np
import ast
from natsort import natsorted

###**Runtime Environment**

In [2]:
curr_dir   = Path().absolute()
os.environ['TZ'] = 'US/Eastern'
time.tzset()
current_time = time.strftime('%a %b %d %H:%M:%S %Y', time.localtime())
print(current_time)
print("\nCurrent directory :{}".format( curr_dir))
print("Python version    :", sys.version.split('\n')[0])

Sat Jan 20 17:48:19 2024

Current directory :/content
Python version    : 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


- map_to_binary.py contains the mapping of labels and body parts
- totalsegmentator_snomed_mapping_with_partial_colors has snomed codes and colors

In [16]:
try:
  os.remove(f'{curr_dir}/map_to_binary.py')
except OSError:
  pass
!wget -q  https://raw.githubusercontent.com/wasserth/TotalSegmentator/v2.0.5/totalsegmentator/map_to_binary.py
!wget -q  https://raw.githubusercontent.com/ImagingDataCommons/CloudSegmentator/main/workflows/TotalSegmentator/resources/totalsegmentator_snomed_mapping_with_partial_colors.csv
import map_to_binary

In [50]:
import ast
import pandas as pd

# Load label ID and body part data
label_body_map = map_to_binary.class_map["total_v1"]
label_body_df = pd.DataFrame(
    list(label_body_map.items()), columns=["labelID", "Structure"]
)

# Function to convert string to list
def str_to_list(s):
    if isinstance(s, list):
        return s
    elif pd.isnull(s):
        return s
    try:
        return ast.literal_eval(s)
    except ValueError:
        return [int(i) for i in s.split(",")]


# Load mapping data
snomed_mapping_df = pd.read_csv(
    "totalsegmentator_snomed_mapping_with_partial_colors.csv",
    encoding="utf-8",
    dtype="str",
)

# Merge dataframes
merged_df = pd.merge(label_body_df, snomed_mapping_df, how="left", on="Structure")

# Assign RGB value to labelID 93
#merged_df.loc[
#    merged_df["labelID"] == "93", "recommendedDisplayRGBValue"
#] = "[255, 182, 193]"

# Convert 'recommendedDisplayRGBValue' from string to list
merged_df["recommendedDisplayRGBValue"] = merged_df["recommendedDisplayRGBValue"].apply(
    str_to_list
)

# Prepare final dataframe
final_df = merged_df.drop(columns=merged_df.filter(regex="Anatomic").columns)
final_df["labelID"] = final_df["labelID"].astype(int)
final_df["SegmentLabel"] = final_df.apply(
    lambda row: row["SegmentedPropertyTypeModifierCodeSequence.CodeMeaning"]
    + " "
    + row["SegmentedPropertyTypeCodeSequence.CodeMeaning"]
    if pd.notnull(row["SegmentedPropertyTypeModifierCodeSequence.CodeMeaning"])
    else row["SegmentedPropertyTypeCodeSequence.CodeMeaning"],
    axis=1,
)
final_df["SegmentDescription"] = (
    "TotalSegmentator "
    + final_df["labelID"].astype(str)
    + " : "
    + final_df["Structure"]
)
final_df = final_df.drop(columns=["Structure"])
final_df["SegmentAlgorithmName"] = "TotalSegmentator v1.5.6"
final_df["SegmentAlgorithmType"] = "AUTOMATIC"
final_df.fillna("", inplace=True)

final_df


,labelID,SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator,SegmentedPropertyCategoryCodeSequence.CodeValue,SegmentedPropertyCategoryCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning,recommendedDisplayRGBValue,SegmentLabel,SegmentDescription,SegmentAlgorithmName,SegmentAlgorithmType
0,1,SCT,123037004,Anatomical Structure,SCT,78961009,Spleen,,,,"[157, 108, 162]",Spleen,TotalSegmentator 1 : spleen,TotalSegmentator v1.5.6,AUTOMATIC
1,2,SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,24028007,Right,"[212, 126, 151]",Right Kidney,TotalSegmentator 2 : kidney_right,TotalSegmentator v1.5.6,AUTOMATIC
2,3,SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,7771000,Left,"[212, 126, 151]",Left Kidney,TotalSegmentator 3 : kidney_left,TotalSegmentator v1.5.6,AUTOMATIC
3,4,SCT,123037004,Anatomical Structure,SCT,28231008,Gallbladder,,,,"[139, 150, 98]",Gallbladder,TotalSegmentator 4 : gallbladder,TotalSegmentator v1.5.6,AUTOMATIC
4,5,SCT,123037004,Anatomical Structure,SCT,10200004,Liver,,,,"[221, 130, 101]",Liver,TotalSegmentator 5 : liver,TotalSegmentator v1.5.6,AUTOMATIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,100,SCT,123037004,Anatomical Structure,SCT,244849004,Deep muscle of back,SCT,7771000,Left,"[192, 104, 88]",Left Deep muscle of back,TotalSegmentator 100 : autochthon_left,TotalSegmentator v1.5.6,AUTOMATIC
100,101,SCT,123037004,Anatomical Structure,SCT,244849004,Deep muscle of back,SCT,24028007,Right,"[192, 104, 88]",Right Deep muscle of back,TotalSegmentator 101 : autochthon_right,TotalSegmentator v1.5.6,AUTOMATIC
101,102,SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,7771000,Left,"[192, 104, 88]",Left Iliopsoas muscle,TotalSegmentator 102 : iliopsoas_left,TotalSegmentator v1.5.6,AUTOMATIC
102,103,SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,24028007,Right,"[192, 104, 88]",Right Iliopsoas muscle,TotalSegmentator 103 : iliopsoas_right,TotalSegmentator v1.5.6,AUTOMATIC


In [51]:
merged_df_json = json.loads(final_df.to_json(orient='records'))

for segment in merged_df_json:
    segment['SegmentedPropertyCategoryCodeSequence'] = {
        'CodingSchemeDesignator': segment.pop('SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator'),
        'CodeValue': segment.pop('SegmentedPropertyCategoryCodeSequence.CodeValue'),
        'CodeMeaning': segment.pop('SegmentedPropertyCategoryCodeSequence.CodeMeaning')
    }
    segment['SegmentedPropertyTypeCodeSequence'] = {
        'CodingSchemeDesignator': segment.pop('SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator'),
        'CodeValue': segment.pop('SegmentedPropertyTypeCodeSequence.CodeValue'),
        'CodeMeaning': segment.pop('SegmentedPropertyTypeCodeSequence.CodeMeaning')
    }
    if ('SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator' in segment and
        (segment['SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator'].strip() or
         segment['SegmentedPropertyTypeModifierCodeSequence.CodeValue'].strip() or
         segment['SegmentedPropertyTypeModifierCodeSequence.CodeMeaning'].strip())):
        segment['SegmentedPropertyTypeModifierCodeSequence'] = {
            'CodingSchemeDesignator': segment.pop('SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator'),
            'CodeValue': segment.pop('SegmentedPropertyTypeModifierCodeSequence.CodeValue'),
            'CodeMeaning': segment.pop('SegmentedPropertyTypeModifierCodeSequence.CodeMeaning')
        }
    else:
        for key in ['SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator',
                    'SegmentedPropertyTypeModifierCodeSequence.CodeValue',
                    'SegmentedPropertyTypeModifierCodeSequence.CodeMeaning']:
            if key in segment:
                del segment[key]
final_json = {
    "BodyPartExamined": "CHEST",
    "ClinicalTrialCoordinatingCenterName": "dcmqi",
    "ClinicalTrialSeriesID": "0",
    "ClinicalTrialTimePointID": "1",
    "ContentCreatorName": "IDC",
    "ContentDescription": "Image segmentation",
    "ContentLabel": "SEGMENTATION",
    "InstanceNumber": "1",
    "SeriesDescription": "TotalSegmentator Segmentation",
    "SeriesNumber": "42",
    "segmentAttributes": [merged_df_json]
}
with open('dicomseg_metadata_whole_slicerAsRef.json', 'w') as file:
    json.dump(final_json, file, indent=4)

In [52]:
import yaml
with open('dicomseg_metadata_whole_slicerAsRef.yaml', 'w') as file:
    yaml.dump(final_json, file)